In [1]:
# Quello che abbiamo fatto fino ad ora è stato usare modelli di LLM per ottenere libri in base
# a concetti di similarità. Ora passiamo alla sezione di sentiiment analysis
# Cerchiamo di assegnare un "sentiment" ai libri a disposizione nel dataset
# Lavorando con testo, possiamo estrarre da questo il sentiment
# Possiamo imparare a filtrare i libri in base a cosa preferisca l'utente (il suo mood)

In [2]:
# In questa sezione rivedremo i modelli di text classification utilizzando i modelli LLM differentemente
# Cosa faremo: divideremo il libri in categorie distinte di sentiment
# 1. Rabbia 
# 2. Disgusto
# 3. Paura
# 4. Gioia
# 5. Tristezza
# 6. Sorpresa
# 7. aggiunta della categoria di sentiment Neutrale, perchè tavolta il testo di un libro
# potrebbe non avere un contesto "emotivo"

# Usando un modello di LLM potremmo estrarre da un libro la sua emozione dominante 
# "A heartwarming journey of love and friendship" -> probabilmente sarà predetto con sentiment "joy"

# Certamente avremmo potuto fare emotion classification con i modelli zero-shot, che siano stati 
# preallenati con abbastanza associazioni tra descrizioni e sentiment analysis
# Utilizzeremo comunque un'altra tecnica, chiamata di fine-tuning per ottere un LLM che faccia questo task

In [3]:
# Rivediamo il modello di RoBERTa, composto da una sequenza di livelli, che fanno l'embedding delle
# parole, poi ci sono i livelli di encoding e gli ultimi strati permettono di fare task di predirre
# la parola mascherata
# Con il fine-tuning, possiamo eliminare questi ultimi strati che fanno predizioni su parole che 
# potrebbero essere sostituite alla parola masked, e sostituirli con strato che predica l'emozione
# Dovremo prendere piccoli dataset contententi descrizioni ed emozioni dominanti associate 
# Con l'applicazione del task di fine-tuning, i modelli non vengono variati, mantengono la loro 
# struttura che si basa su training su linguaggio naturale, ma possiamo invece prendere queste informazioni
# e quelle presenti nel dataset emotion e lasciare che il modello impari i nuovi pesi nel suo layer finale
# Si otterrà un modello in grado di fare emotion classification

# In questo caso utilizzeremo un modello che è già stato fine-tuned ed è stato caricato su Hugginface hub

In [4]:
import pandas as pd

books = pd.read_csv("books_with_categories.csv")

In [5]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883: A NOVEL THAT READERS and critic...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982: A new 'Christie for Christmas' ...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736: A memorable, mesmerizing heroin...",Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897: Lewis' work on the nature of lo...,Nonfiction
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934: ""In The Problem of Pain, C.S. L...",Nonfiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222: On A Train Journey Home To Nort...,Fiction
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014: This book tells the tale of a m...,Nonfiction
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623: Wisdom to Create a Life of Pass...,Fiction
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535: This collection of the timeless...,Nonfiction


In [6]:
# Come nella sezione precedente, andiamo in Huggingface models, scegliamo text classification, 
# scriviamo nella barra di ricerca "emotion" e il modello più in trend è "SamLowe/roberta-base-go_emotions"
# E' un fine-tuned di RoBERTa e ha tante emozioni, forse anche con troppe emotion
# Guardiamo quindi il modello sottostante nella classifica in trend, e vediamo "j-hartmann/emotion-english-distilroberta-base"
# è stato fine-tuned su 7 emozioni base (quelle che erano state decise sopra)
# Copiamo quello che nella pagina è mostrato come la base per partire con questo modello
from transformers import pipeline

classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
classifier("I love this!")

Device set to use cpu
C:\Users\mazze\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'anger', 'score': 0.004419779404997826},
  {'label': 'disgust', 'score': 0.0016119893407449126},
  {'label': 'fear', 'score': 0.0004138521908316761},
  {'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'neutral', 'score': 0.005764586851000786},
  {'label': 'sadness', 'score': 0.002092391485348344},
  {'label': 'surprise', 'score': 0.00852868054062128}]]

In [7]:
# Andando a vedere al sito "dataloop" come performa questo modello, vediamo che è buon modello
# Infatti, essendo fine-tuned, possiamo vedere come performa sulle emotion definite
# E' scritto che ottiene come accuratezza circa il 66%, che potrebbe sembrare basso, ma supera di 
# molto la percentuale del 14% per la random-chance baseline
# Usiamo allora questo modello per fare estrapolazione di sentiment dalle descrizioni dei nostri libri

In [8]:
# Dal risultato del lanciare il codice sopra del modello, si ottengono due warning in particolare
# Il primo, sull'utilizzo di una gpus ma potremmo usarla
# Secondo punto, l'argomento "return_all_scores" che era nella documentazione è ora deprecato, e
# modifichiamo quindi la funzione come indicato
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    top_k=None
    #, device="" # Potrei mettere la gpu
)
classifier("I love this!")

Device set to use cpu


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.00852868054062128},
  {'label': 'neutral', 'score': 0.005764586851000786},
  {'label': 'anger', 'score': 0.004419779404997826},
  {'label': 'sadness', 'score': 0.002092391485348344},
  {'label': 'disgust', 'score': 0.0016119893407449126},
  {'label': 'fear', 'score': 0.0004138521908316761}]]

In [9]:
# Ora dobbiamo capire a cosa applicare il sentimenti analysis
# Potremmo applicarlo in maniera globale alla descrizione, oppure studiare pezzi separati di questa
# Per esempio, guardiamo la prima descrizione
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [10]:
# Proviamo a capire il sentiment di ogni frase:

# 1. "A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives"
# Quasi potremmo dire "surpising"

# 2. "Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist."
# "fear"

# Vediamo già che assegnare una unica etichetta di sentiment non è un compito semplice, dare
# un'unica caratterizzazione farebbe anche perdere informazioni su quel libro

In [11]:
# Facciamo una prova: facciamo partire il sentiment analysis sull'intera descrizione
classifier(books["description"][0])

# Rileggendo il testo ci sono delle frasi anche di gioia, ma questo non viene catturato
# Come facciamo quindi a gestire la situazione?

[[{'label': 'fear', 'score': 0.6548405885696411},
  {'label': 'neutral', 'score': 0.16985228657722473},
  {'label': 'sadness', 'score': 0.11640921980142593},
  {'label': 'surprise', 'score': 0.02070065587759018},
  {'label': 'disgust', 'score': 0.019100677222013474},
  {'label': 'joy', 'score': 0.01516144908964634},
  {'label': 'anger', 'score': 0.003935146611183882}]]

In [12]:
# Possiamo suddividere la descrizione in sentenze individuali e classificare queste
# Per fortuna, il modello permette anche una classificazione multima contemporaneamente
# Quindi, scriviamo più o meno lo stesso argomento, però facendo lo split alla fine di ogni sentenza
classifier(books["description"][0].split("."))

# Come si nota, ci sono molte più elaborazioni diverse di emozioni, la prima frase ad esempio
# risulta essere di sorpresa, come era stato pensato
# Questo processo è molto più accurato e da una maggiore varietà di emozioni al testo

[[{'label': 'surprise', 'score': 0.7296020984649658},
  {'label': 'neutral', 'score': 0.14038600027561188},
  {'label': 'fear', 'score': 0.06816228479146957},
  {'label': 'joy', 'score': 0.04794260859489441},
  {'label': 'anger', 'score': 0.009156366810202599},
  {'label': 'disgust', 'score': 0.0026284765917807817},
  {'label': 'sadness', 'score': 0.002122163539752364}],
 [{'label': 'neutral', 'score': 0.44937002658843994},
  {'label': 'disgust', 'score': 0.27359163761138916},
  {'label': 'joy', 'score': 0.10908330976963043},
  {'label': 'sadness', 'score': 0.09362746775150299},
  {'label': 'anger', 'score': 0.04047830402851105},
  {'label': 'surprise', 'score': 0.026970159262418747},
  {'label': 'fear', 'score': 0.006879047024995089}],
 [{'label': 'neutral', 'score': 0.6462159752845764},
  {'label': 'sadness', 'score': 0.24273329973220825},
  {'label': 'disgust', 'score': 0.04342271760106087},
  {'label': 'surprise', 'score': 0.028300564736127853},
  {'label': 'joy', 'score': 0.014211

In [13]:
# Facciamo giusto una prova: vediamo se l'emozione associata ad un testo è esatta
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [14]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [15]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296020984649658},
 {'label': 'neutral', 'score': 0.14038600027561188},
 {'label': 'fear', 'score': 0.06816228479146957},
 {'label': 'joy', 'score': 0.04794260859489441},
 {'label': 'anger', 'score': 0.009156366810202599},
 {'label': 'disgust', 'score': 0.0026284765917807817},
 {'label': 'sadness', 'score': 0.002122163539752364}]

In [16]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [17]:
predictions[3]

[{'label': 'fear', 'score': 0.9281681180000305},
 {'label': 'anger', 'score': 0.032191041857004166},
 {'label': 'neutral', 'score': 0.01280867587774992},
 {'label': 'sadness', 'score': 0.008756861090660095},
 {'label': 'surprise', 'score': 0.008597906678915024},
 {'label': 'disgust', 'score': 0.008431818336248398},
 {'label': 'joy', 'score': 0.0010455837473273277}]

In [18]:
sentences[5]

' Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world has to offer'

In [19]:
predictions[5]

[{'label': 'joy', 'score': 0.932798445224762},
 {'label': 'disgust', 'score': 0.0377168171107769},
 {'label': 'neutral', 'score': 0.015891803428530693},
 {'label': 'sadness', 'score': 0.006444480270147324},
 {'label': 'anger', 'score': 0.005024945829063654},
 {'label': 'surprise', 'score': 0.0015812049387022853},
 {'label': 'fear', 'score': 0.0005423048860393465}]

In [20]:
# Questo fatto introduce però una difficoltà, legata alla presenza di multiple emotion ai libri
# Un approccio potrebbe essere quello di avere per ogni libro 7 colonne relative alle 7 emozioni
# e possiamo mostrare la probabilità legata ad ogni emozione relativa a quel testo
# Prendiamo solo il massimo score relativo ad ogni emozione
predictions

[[{'label': 'surprise', 'score': 0.7296020984649658},
  {'label': 'neutral', 'score': 0.14038600027561188},
  {'label': 'fear', 'score': 0.06816228479146957},
  {'label': 'joy', 'score': 0.04794260859489441},
  {'label': 'anger', 'score': 0.009156366810202599},
  {'label': 'disgust', 'score': 0.0026284765917807817},
  {'label': 'sadness', 'score': 0.002122163539752364}],
 [{'label': 'neutral', 'score': 0.44937002658843994},
  {'label': 'disgust', 'score': 0.27359163761138916},
  {'label': 'joy', 'score': 0.10908330976963043},
  {'label': 'sadness', 'score': 0.09362746775150299},
  {'label': 'anger', 'score': 0.04047830402851105},
  {'label': 'surprise', 'score': 0.026970159262418747},
  {'label': 'fear', 'score': 0.006879047024995089}],
 [{'label': 'neutral', 'score': 0.6462159752845764},
  {'label': 'sadness', 'score': 0.24273329973220825},
  {'label': 'disgust', 'score': 0.04342271760106087},
  {'label': 'surprise', 'score': 0.028300564736127853},
  {'label': 'joy', 'score': 0.014211

In [21]:
# Abbiamo un diverso ordine delle emozioni per ogni frase in base all'importanza per la sentenza specifica
# Le ordiniamo tutte allo stesso modo per leggerle meglio
sorted(predictions[0], key=lambda x: x["label"])

[{'label': 'anger', 'score': 0.009156366810202599},
 {'label': 'disgust', 'score': 0.0026284765917807817},
 {'label': 'fear', 'score': 0.06816228479146957},
 {'label': 'joy', 'score': 0.04794260859489441},
 {'label': 'neutral', 'score': 0.14038600027561188},
 {'label': 'sadness', 'score': 0.002122163539752364},
 {'label': 'surprise', 'score': 0.7296020984649658}]

In [22]:
# Cerchiamo di ottenere il punteggio massimo per ogni emozione 
# Creiamo la lista "emotion_labels" che saranno le key per il dizionario, che diventeranno
# le colonne nel dataframe
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]
isbns = [] # La creiamo per poi mergare il dataframe con il dataset iniziale
emotion_scores = {label: [] for label in emotion_labels} # Manterrà lo score per ogni description

# La funzione calcola lo score massimo per ogni descrizione
def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [23]:
# Ora, per ogni descrizione abbiamo un dizionario contenente le probabilità massime per ogni emotion
# Applichiamo la funzione per ogni libro (facciamo prima una prova per pochi libri)
for i in range(10):
    isbns.append(books["isbn13"][i]) # Memorizziamo il codice "isbn13" per ogni libro che si sta trattando
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions) # Prendiamo i valori massimi
    
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label]) # Appendiamo tutto al dizionario emotion_scores

In [24]:
# Dovremmo avere un dizionario che servirà come base per il nostro dataframe contenente diverse
# colonne con le probabilità massime per ogni libro
emotion_scores

{'anger': [0.0641336441040039,
  0.6126197576522827,
  0.0641336441040039,
  0.35148438811302185,
  0.08141235262155533,
  0.2322252243757248,
  0.5381842255592346,
  0.0641336441040039,
  0.3006700277328491,
  0.0641336441040039],
 'disgust': [0.27359163761138916,
  0.3482847511768341,
  0.10400667786598206,
  0.1507224589586258,
  0.18449543416500092,
  0.7271744608879089,
  0.155854731798172,
  0.10400667786598206,
  0.2794816195964813,
  0.17792661488056183],
 'fear': [0.9281681180000305,
  0.9425276517868042,
  0.9723208546638489,
  0.3607059419155121,
  0.09504334628582001,
  0.05136283114552498,
  0.7474274635314941,
  0.4044976532459259,
  0.9155241250991821,
  0.05136283114552498],
 'joy': [0.932798445224762,
  0.7044219970703125,
  0.7672380805015564,
  0.25188151001930237,
  0.04056443274021149,
  0.043375786393880844,
  0.872565746307373,
  0.04056443274021149,
  0.04056443274021149,
  0.04056443274021149],
 'neutral': [0.6462159752845764,
  0.8879395127296448,
  0.54947674

In [25]:
# Avendo visto ora che funziona, lo facciamo per tutti i libri
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]
isbns = [] 
emotion_scores = {label: [] for label in emotion_labels} 

for i in tqdm(range(len(books))):
    isbns.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [32:20<00:00,  2.68it/s]


In [27]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbns

In [28]:
# Abbiamo tante colonne per quante sono le emozioni e compariranno per ogni campione (riga)
# il valore massimo di probabilità per quella emozione estrapolata da righe della descrizione
emotions_df.head()

,anger,disgust,fear,joy,neutral,sadness,surprise,isbn13
0,0.064134,0.273592,0.928168,0.932798,0.646216,0.967158,0.729602,9780002005883
1,0.612620,0.348285,0.942528,0.704422,0.887940,0.111690,0.252546,9780002261982
2,0.064134,0.104007,0.972321,0.767238,0.549477,0.111690,0.078766,9780006178736
3,0.351484,0.150722,0.360706,0.251882,0.732684,0.111690,0.078766,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884390,0.475880,0.078766,9780006280934


In [29]:
books = pd.merge(books, emotions_df, on="isbn13")

In [30]:
# Possiamo anche vedere la distribuzione delle emozioni (studiando singolarmente le colonne delle 
# emozioni) e possiamo vedere che c'è una buona rappresentazione
# In particolare, "sadness" ha molte probabilità alte, così come le altre emozioni, e quindi
# potremmo in qualche modo mescolare i libri in modo ottimale al momento della raccomandazione agli utenti
# Salviamo quindi come ultimo step il nuovo dataset
books.to_csv("books_with_emotions.csv", index=False)

In [31]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,neutral,sadness,surprise
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883: A NOVEL THAT READERS and critic...,Fiction,0.064134,0.273592,0.928168,0.932798,0.646216,0.967158,0.729602
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982: A new 'Christie for Christmas' ...,Fiction,0.612620,0.348285,0.942528,0.704422,0.887940,0.111690,0.252546
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736: A memorable, mesmerizing heroin...",Fiction,0.064134,0.104007,0.972321,0.767238,0.549477,0.111690,0.078766
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897: Lewis' work on the nature of lo...,Nonfiction,0.351484,0.150722,0.360706,0.251882,0.732684,0.111690,0.078766
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934: ""In The Problem of Pain, C.S. L...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884390,0.475880,0.078766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222: On A Train Journey Home To Nort...,Fiction,0.148208,0.030643,0.919165,0.255172,0.853721,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014: This book tells the tale of a m...,Nonfiction,0.064134,0.114383,0.051363,0.400262,0.883198,0.111690,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623: Wisdom to Create a Life of Pass...,Fiction,0.009997,0.009929,0.339218,0.947779,0.375754,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535: This collection of the timeless...,Nonfiction,0.064134,0.104007,0.459269,0.759457,0.951104,0.368111,0.078766
